# Предсказание уровня активности аэропортов в Бразилии,
## Постановка задачи:

Исходные данные:

In [ ]:
import sys
sys.path.append('../')
from stable_gnn.graph import Graph
from stable_gnn.pipelines.node_classification_pipeline import TrainModelNC, TrainModelOptunaNC

import torch_geometric.transforms as T
import torch



In [ ]:
root = '../data_validation/'
name='wisconsin'
dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(),adjust_flag=False)
data = dataset[0]
len(data.edge_index[0])

Решаем задачу предсказания связей, пользуясь подготовленным пайплайном в stable_gnn.pipelines.train_model_pipeline

In [ ]:
data.num_nodes

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss_name='APP'
ssl_flag = False

params = {'hidden_layer': 128, "size of network, number of convs": 3, "dropout": 0.1, "lr": 0.01, "coef": 10}
model_training = TrainModelNC(data=dataset, device=device, ssl_flag=ssl_flag, loss_name=loss_name)
model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(params, plot_training_procces=True)
print(train_acc_mi, test_acc_mi)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss_name='APP'
ssl_flag = True

params = {'hidden_layer': 64, "size of network, number of convs": 3, "dropout": 0.1, "lr": 0.01, "coef":10}
model_training = TrainModelNC(data=dataset, device=device, ssl_flag=ssl_flag, loss_name=loss_name)
model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(params, plot_training_procces=True)
print(train_acc_mi, test_acc_mi)

# Если уточнить структуру
удалим датасет и загрузим снова с другим флагом:

In [ ]:
root = '../data_validation/'
name='brazil'
dataset = Graph(root=root + str(name), name=name, transform=T.NormalizeFeatures(),adjust_flag=True,number_of_trainig_epochs=118)
data = dataset[0]

print(len(data.edge_index[0]))

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss_name='APP'
ssl_flag = False

params = {'hidden_layer': 128, "size of network, number of convs": 3, "dropout": 0.1, "lr": 0.01, "coef":10}
model_training = TrainModelNC(data=dataset, device=device, ssl_flag=ssl_flag, loss_name=loss_name)
model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(params, plot_training_procces=True)
print(train_acc_mi, test_acc_mi)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

loss_name='APP'
ssl_flag = True

params = {'hidden_layer': 64, "size of network, number of convs": 3, "dropout": 0.1, "lr": 0.01, "coef": 10}
model_training = TrainModelNC(data=dataset, device=device, ssl_flag=ssl_flag, loss_name=loss_name)
model, train_acc_mi, train_acc_ma, test_acc_mi, test_acc_ma = model_training.run(params, plot_training_procces=True)
print(train_acc_mi, test_acc_mi)

# Построим объяснения предсказания

In [ ]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric.utils import to_dense_adj

from stable_gnn.explain import Explain

root='../data_validation/'
features = np.load(root + name + "/X.npy")
if os.path.exists(root + name + "/A.npy"):
    adj_matrix = np.load(root + name + "/A.npy")
else:
    adj_matrix = torch.squeeze(to_dense_adj(data.edge_index.cpu())).numpy()


explainer = Explain(model=model, adj_matrix=adj_matrix, features=features)

for global_node in [0,7]:
        pgm_explanation = explainer.structure_learning(global_node)
        print("explanations is", global_node, pgm_explanation.nodes, pgm_explanation.edges)
        g = nx.DiGraph()
        mapping = {}
        inv_mapping = {}
        for i, node in enumerate(pgm_explanation.nodes):
            mapping[node]=i
            inv_mapping[i]=node

        edges = []
        for edge in pgm_explanation.edges:
            edges.append([mapping[edge[0]], mapping[edge[1]]])
        g.add_edges_from(edges)
        for node in g.nodes():
            print(node)
            g.add_node(node)

        plt.figure()
        nx.draw(g)
        plt.title(str(global_node)+'\'s explanation' )
        plt.show()
        plt.close()

# Ego-networks of these nodes

In [ ]:
edges = data.edge_index.T.tolist()
G = nx.Graph()
G.add_edges_from(edges)
G.remove_edges_from(nx.selfloop_edges(G))

for global_node in [0,7]:
    g = nx.ego_graph(G,global_node)
    plt.figure()
    nx.draw(g)
    plt.title(str(global_node)+'\'s ego-network' )
    plt.show()
    plt.close()